## How to create a closed-loop optimisation using the single objective Ackley synthetic function as the benchmark function

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

### Setting up the variables and objectives for the synthetic function

The following table shows the ideal parameters to use for each single objective synthetic function, where n_var is the number of variables

| Name | Maximise/Minimise | String | Number of variables | Variable bounds | Objective bounds |
| --- | --- | --- | --- | --- | --- |
| Ackley | Minimise | "ackley" | any |all [-32.768, 32.768] | [0.0, 25.0] |
| Griewank | Minimise | "griewank" | any | all [-600.0, 600.0] | [0.0, 100.0 * n_var] |
| Levy | Minimise | "levy" | any | all [-10.0, 10.0] | [0.0, 50.0 * n_var] |
| Michalewicz | Minimise | "michalewicz" | 2, 5, or 10 | all [0.0, math.pi] | [-1.9, 0.0], [-4.8, 0.0], or [-9.8, 0.0] |
| Rastrigin | Minimise | "rastrigin" | any | all [-5.12, 5.12] | [0.0, 50.0 * n_var] |
| Rosenbrock | Minimise | "rosenbrock" | any | all [-5.0, 10.0] | [0.0, 810081.0 * (n_var - 1)] |
| Styblinski-Tang | Minimise | "styblinski-tang" | any | all [-5.0, 5.0] | [-40.0 * n_var, 25.0] |

The number of variable objects need to be created according to the specified described in the table above.

The example below uses the Ackley function with three variables

In [ ]:
# Create the variable objects
var_list = VariablesList(
    [ContinuousVariable(name=f"var{n}", lower_bound=-5.0, upper_bound=5.0, units="") for n in range(1, 4)]
)

In [ ]:
# Create the objective objects
obj_list = ObjectivesList(
    [RegressionObjective(name="Ackley_obj1", obj_max_bool=False, lower_bound=0.0, upper_bound=25.0)]
)

In [ ]:
# Instantiate the sampler
sampler = LatinHyperCubeSampling()

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

### Setting up the benchmark

The `SingleObjectiveSyntheticBenchmark` class is instantiated with the string name of the synthetic function and the number of variables as arguments and the size of the noise for the function as a keyword argument

In [ ]:
from nemo_bo.opt.benchmark import MultiObjectiveSyntheticBenchmark

# Instantiate the MultiObjectiveSyntheticBenchmark class to be the benchmark function
benchmark = SingleObjectiveSyntheticBenchmark("ackley", var_list.n_var, noise_std=0.25)

In [ ]:
# Set up the optimisation instance whilst passing benchmark function
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler, benchmark_func=benchmark)

### Starting the optimisation run with the benchmark function

When using a benchmark function in an optimisation, X and Y arrays do not need to be passed. The training set will be automatically created, where the number of samples is related to the sampler type chosen and the number of variables (`number of samples = (2 * number of variables) + 2`)

When the plot_progress keyword argument is True, a pareto plot will be created at every iteration.

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
optimisation_data = optimisation.run(number_of_iterations=50, plot_progress=True)